In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from time import sleep
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
secrets_file = open("secrets.txt","r")

In [3]:
string = secrets_file.read()

In [6]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [8]:

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [9]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3)) 
    return tracks

In [10]:
# all_tracks = get_playlist_tracks("1G8IpkZKobrIlXcVPoSIuf")
# len(all_tracks)

10000

In [87]:
all_tracks [0]['track']['name']

'Like a Rolling Stone'

In [49]:
all_tracks[0]["track"]["album"]['artists'][0]['name']

'Bob Dylan'

In [67]:
all_tracks[2]['track']['uri']

'spotify:track:3ZFBeIyP41HhnALjxWy1pR'

In [186]:
# some of the tracks dont have name, artistname and uri and so when writing the for loop we 
# to do try and except and put 'none'

all_tracks[467]

{'added_at': '2020-11-29T15:02:07Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/acclaimedmusic'},
  'href': 'https://api.spotify.com/v1/users/acclaimedmusic',
  'id': 'acclaimedmusic',
  'type': 'user',
  'uri': 'spotify:user:acclaimedmusic'},
 'is_local': False,
 'primary_color': None,
 'track': None,
 'video_thumbnail': {'url': None}}

In [68]:
song_name = []
artist_name = []
track_uri = []

for i in range(len(all_tracks)):
    try:
        song = all_tracks[i]['track']['name']
        artist = all_tracks[i]["track"]["album"]['artists'][0]['name']
        uri = all_tracks[i]['track']['uri']
        
        artist_name.append(artist)
        song_name.append(song)
        track_uri.append(uri)
    except:
        artist_name.append('none')
        song_name.append('none')
        track_uri.append('none')

In [74]:
# making data frame

spotify_df = pd.DataFrame({'song' : song_name,
                          'artist': artist_name,
                          'uri': track_uri})

In [88]:
spotify_df.head()

,song,artist,uri
0,Like a Rolling Stone,Bob Dylan,spotify:track:3AhXZa8sUQht0UEdBJgpGc
1,Smells Like Teen Spirit,Nirvana,spotify:track:3oTlkzk1OtrhH8wBAduVEi
2,A Day In The Life - Remastered,The Beatles,spotify:track:3ZFBeIyP41HhnALjxWy1pR
3,Good Vibrations (Mono),The Beach Boys,spotify:track:5Qt4Cc66g24QWwGP3YYV9y
4,Johnny B Goode,Chuck Berry,spotify:track:7MH2ZclofPlTrZOkPzZKhK


In [90]:
spotify_df.song.value_counts() # 53 songs have no name

none                                           53
Summertime                                      6
Crazy                                           4
Fire                                            4
Girlfriend                                      4
                                               ..
First Time I Met The Blues - Single Version     1
Complete Control                                1
Don't You Know?                                 1
Ten Cents a Dance                               1
The Man Machine - 2009 Remaster                 1
Name: song, Length: 9641, dtype: int64

In [147]:
spotify_df = spotify_df[spotify_df.song != 'none'] # removing all the rows that have 'none'

In [167]:
spotify_df.shape

(9947, 4)

In [166]:
# since we removed all the values that had 'none' we need to reset index

spotify_df.reset_index(inplace=True)  

In [172]:
spotify_df.drop('index', axis=1, inplace=True) # drop extra index column created after reset_index

In [173]:
spotify_df.head()

,song,artist,uri
0,Like a Rolling Stone,Bob Dylan,spotify:track:3AhXZa8sUQht0UEdBJgpGc
1,Smells Like Teen Spirit,Nirvana,spotify:track:3oTlkzk1OtrhH8wBAduVEi
2,A Day In The Life - Remastered,The Beatles,spotify:track:3ZFBeIyP41HhnALjxWy1pR
3,Good Vibrations (Mono),The Beach Boys,spotify:track:5Qt4Cc66g24QWwGP3YYV9y
4,Johnny B Goode,Chuck Berry,spotify:track:7MH2ZclofPlTrZOkPzZKhK


# Audio Features

In [98]:
sp.audio_features(track_uri[1])

[{'danceability': 0.485,
  'energy': 0.863,
  'key': 1,
  'loudness': -9.027,
  'mode': 1,
  'speechiness': 0.0495,
  'acousticness': 1.22e-05,
  'instrumentalness': 0.0162,
  'liveness': 0.138,
  'valence': 0.767,
  'tempo': 116.835,
  'type': 'audio_features',
  'id': '3oTlkzk1OtrhH8wBAduVEi',
  'uri': 'spotify:track:3oTlkzk1OtrhH8wBAduVEi',
  'track_href': 'https://api.spotify.com/v1/tracks/3oTlkzk1OtrhH8wBAduVEi',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3oTlkzk1OtrhH8wBAduVEi',
  'duration_ms': 300977,
  'time_signature': 4}]

In [107]:
# collecting all the audiofeatures in a list
# commenting this code to prevent running the spotify api

# audio_features = []
# counter = 0  # setting a counter so that we can stop the code for 10 seconds when counter is 100

# for item in track_uri:
#     features = sp.audio_features(item)
#     audio_features.append(features)

#     counter = counter + 1
    
#     if counter % 100 == 0:
#         sleep(10)


In [184]:
[None] in audio_features # some of the audiofeatures have [None] because there was no trackuri

True

In [176]:
danceability = []
energy = []
loudness = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []


for i in range(len(audio_features)):
    #making sure we only run the for loop when it is not [None]
    if audio_features[i] != [None]:  
        danceability.append(audio_features[i][0]['danceability'])
        energy.append(audio_features[i][0]['energy'])
        loudness.append(audio_features[i][0]['loudness'])
        speechiness.append(audio_features[i][0]['speechiness'])
        acousticness.append(audio_features[i][0]['acousticness'])
        instrumentalness.append(audio_features[i][0]['instrumentalness'])
        liveness.append(audio_features[i][0]['liveness'])
        valence.append(audio_features[i][0]['valence'])
        tempo.append(audio_features[i][0]['tempo'])


In [177]:
spotify_af = pd.DataFrame({'danceability' : danceability,
                          'energy': energy,
                          'loudness': loudness,
                          'speechiness': speechiness,
                           'acousticness': acousticness,
                           'instrumentalness': instrumentalness,
                           'liveness': liveness,
                           'valence': valence,
                           'tempo': tempo
                          })

In [178]:
spotify_af

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.482,0.721,-6.839,0.0321,0.731000,0.000000,0.1890,0.557,95.263
1,0.485,0.863,-9.027,0.0495,0.000012,0.016200,0.1380,0.767,116.835
2,0.364,0.457,-14.162,0.0675,0.290000,0.000106,0.9220,0.175,163.219
3,0.398,0.413,-10.934,0.0388,0.082200,0.000025,0.0891,0.331,133.574
4,0.518,0.756,-10.851,0.0915,0.735000,0.000062,0.3170,0.968,166.429
...,...,...,...,...,...,...,...,...,...
9942,0.332,0.707,-12.698,0.0361,0.000012,0.006120,0.1100,0.652,144.815
9943,0.464,0.749,-8.564,0.4800,0.224000,0.000046,0.3510,0.879,181.121
9944,0.694,0.710,-9.793,0.3400,0.527000,0.001350,0.0697,0.415,119.964
9945,0.332,0.800,-9.746,0.0326,0.000368,0.001600,0.0850,0.832,149.240


In [179]:
spotify_final = pd.concat([spotify_df, spotify_af], axis=1)

In [181]:
spotify_final.head()

,song,artist,uri,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Like a Rolling Stone,Bob Dylan,spotify:track:3AhXZa8sUQht0UEdBJgpGc,0.482,0.721,-6.839,0.0321,0.731000,0.000000,0.1890,0.557,95.263
1,Smells Like Teen Spirit,Nirvana,spotify:track:3oTlkzk1OtrhH8wBAduVEi,0.485,0.863,-9.027,0.0495,0.000012,0.016200,0.1380,0.767,116.835
2,A Day In The Life - Remastered,The Beatles,spotify:track:3ZFBeIyP41HhnALjxWy1pR,0.364,0.457,-14.162,0.0675,0.290000,0.000106,0.9220,0.175,163.219
3,Good Vibrations (Mono),The Beach Boys,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,0.398,0.413,-10.934,0.0388,0.082200,0.000025,0.0891,0.331,133.574
4,Johnny B Goode,Chuck Berry,spotify:track:7MH2ZclofPlTrZOkPzZKhK,0.518,0.756,-10.851,0.0915,0.735000,0.000062,0.3170,0.968,166.429


In [182]:
spotify_final.to_csv('spotify_final.csv')